## This notebook assumes that you have already tuned the hyperparameters of the model using the gridSearch notebook, or, know the best hyperparameters to use for your data.

In [9]:
import numpy as np
import pandas as pd
import sys
import os 
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
import glob

from sglm import utils, glm_fit, predictor_labels

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create a project

#### First, let's create a new project. The project directory will create a data and results folder and a config file.

#### You will need to edit the config file with the particular glm params you wish to use. Fields that are necessary to edit are: predictors, predictors_shift_bounds, response, and the glm_keyword_args.

#### You will also need to move your data into the data folder.

In [10]:
#set data path
workingDirectory = os.getcwd()
dataFolder = 'projects'
workingPath = os.path.join(workingDirectory, dataFolder)
print(workingPath)

e:\Emily\sabatini-glm-workflow\projects


In [11]:
project_name = 'PV_combinedCatchx4_Lhemi_bestFit'
project_dir = workingPath

utils.create_new_project(project_name, project_dir)

Project directory already exists!


'e:\\Emily\\sabatini-glm-workflow\\projects\\PV_combinedCatchx4_Lhemi_bestFit\\config.yaml'

In [12]:
#set analysis parameters in the config file
#predictor_list = ['go','nogo','lick_1R','lick_1','lick_2','lick_3','last_lick','lick_non1-3','missReward']
predictor_list = ['go','nogo','lick_1','lick_2','lick_3','lick_3R','lick_3C','last_lick','lick_non1-3','missReward','freeReward']
predictors_shift_bounds_list = [-50, 100]
response = 'photometryLhemi'
regression_type = 'ElasticNet'
alpha = 1e-05
l1_ratio = 0.75

# Import and Format Data

Input data should conform to the following convention and be saved as a *.csv:

Indices / Unique Row Identifiers:
* SessionName -- Any order is acceptable
* TrialNumber-- Must be in chronological order, but does not need to start from zero
* Timestamp -- Must be in chronological order, but does not need to start from zero

Columns (Predictors + Responses):
* Predictors - binary
* Reponses - e.g. neural responses (analog or binary)

Example, shown below is dummy data depicting a trial_0 that last four response timestamps:
| SessionName | TrialNumber | Timestamp | predictor_1 | predictor_2 | predictor_3 | response_1 | response_2 |
| --- | --- | --- | --- | --- | --- | --- | --- |
| session_0 | trial_0 | -1 | 0 | 0 | 0 | 1 | 0.3 |
| session_0 | trial_0 | 0 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_0 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_0 | 2 | 0 | 1 | 0 | 1 | 0.3 |
| session_0 | trial_1 | -2 | 0 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | -1 | 0 | 0 | 0 | 1 | 2.3 |
| session_0 | trial_1 | 0 | 1 | 0 | 0 | 0 | 1.4 |
| session_0 | trial_1 | 1 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 5 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 6 | 1 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_0 | 7 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_0 | 8 | 0 | 0 | 0 | 1 | 2.3 |
| session_1 | trial_1 | 9 | 0 | 0 | 0 | 0 | 1.4 |
| session_1 | trial_1 | 10 | 0 | 0 | 0 | 1 | 2.3 |
....

#### Now, let's get set up to start our project

In [13]:
project_path = os.path.join(project_dir, project_name)
files = os.listdir(project_path)

assert 'data' in files, 'data folder not found! {}'.format(files)
assert 'results' in files, 'results folder not found! {}'.format(files)
assert 'config.yaml' in files, 'config.yaml not found! {}'.format(files)

#### If needed, use the following function to combine multiple sessions into one csv. You will need a filename you wish to call your output_csv

In [14]:
output_csv = 'combined.csv'

utils.combine_csvs(project_path, output_csv)

Output file already exists! Please remove or rename the existing file: e:\Emily\sabatini-glm-workflow\projects\PV_combinedCatchx4_Lhemi_bestFit\data\combined.csv


'e:\\Emily\\sabatini-glm-workflow\\projects\\PV_combinedCatchx4_Lhemi_bestFit\\data\\combined.csv'

#### Next, we'll load the data and set the columns you wish to use as fixed indices

In [15]:
input_file = os.path.join(project_path, 'data', output_csv)
index_col = ['SessionName', 'TrialNumber', 'Timestamp']

df = utils.read_data(input_file, index_col)

print('Your dataframe has {} rows and {} columns'.format(df.shape[0], df.shape[1]))

Your dataframe has 1542488 rows and 7 columns


#### You can now explore and add to the dataframe. As an example, you may want to add various "predictors" or "features" to explore. You can use the example below as inspiration

#### Friendly reminder, the df we have imported is mutli-index, meaning, it's organization is dependent on 3-columns that we have set in index_col. Therefore, we can use "groupby" if you need to split the organization. 

In [16]:
#Define predictor labels
df_source = predictor_labels.predictor_labels(df)
df_source

go  nogo  lick  reward outcome  \
SessionName       TrialNumber Timestamp                                   
20231206_AB100_g0 0           0           0     0     0       0     NaN   
                              1           0     0     0       0     NaN   
                              2           0     0     0       0     NaN   
                              3           0     0     0       0     NaN   
                              4           0     0     0       0     NaN   
...                                      ..   ...   ...     ...     ...   
20240119_AB95_g0  278         117209      0     0     0       0       H   
                              117210      0     0     0       0       H   
                              117211      0     0     0       0       H   
                              117212      0     0     0       0       H   
                              117213      0     0     0       0       H   

                                         photometryLhemi  photometryRhemi  \
SessionName       TrialNumber Timestamp                                     
20231206_AB100_g0 0           0                -0.252277        -0.653752   
                              1                -0.381614        -0.869515   
                              2                -0.445497        -1.002617   
                              3                -0.536251        -1.227449   
                              4                -0.560333        -1.174072   
...                                                  ...              ...   
20240119_AB95_g0  278         117209           -0.331894         0.037879   
                              117210           -0.227099        -0.358915   
                              117211           -0.210129         0.255514   
                              117212           -0.133035         0.123817   
                              117213           -0.127012        -0.464694   

                                         lick_1  lick_2  lick_3  ...  go_hit  \
SessionName       TrialNumber Timestamp                          ...           
20231206_AB100_g0 0           0               0       0       0  ...       0   
                              1               0       0       0  ...       0   
                              2               0       0       0  ...       0   
                              3               0       0       0  ...       0   
                              4               0       0       0  ...       0   
...                                         ...     ...     ...  ...     ...   
20240119_AB95_g0  278         117209          0       0       0  ...       0   
                              117210          0       0       0  ...       0   
                              117211          0       0       0  ...       0   
                              117212          0       0       0  ...       0   
                              117213          0       0       0  ...       0   

                                         go_miss  nogo_cr  nogo_fa  lick_1R  \
SessionName       TrialNumber Timestamp                                       
20231206_AB100_g0 0           0                0        0        0        0   
                              1                0        0        0        0   
                              2                0        0        0        0   
                              3                0        0        0        0   
                              4                0        0        0        0   
...                                          ...      ...      ...      ...   
20240119_AB95_g0  278         117209           0        0        0        0   
                              117210           0        0        0        0   
                              117211           0        0        0        0   
                              117212           0        0        0        0   
                              117213           0        0        0        

In [17]:
#Confirm that the unique trials are equal to the expected numbers
filtered_df = df_source[df_source['go']==1]
unique_trial_numbers = pd.unique(filtered_df.index.get_level_values('TrialNumber'))
unique_trial_numbers
print(len(unique_trial_numbers))
filtered_df

292


go  nogo  lick  reward outcome  \
SessionName       TrialNumber Timestamp                                   
20231206_AB100_g0 1           125         1     0     0       0       H   
                  4           2044        1     0     0       0       H   
                  6           3394        1     0     0       0       H   
                  8           4949        1     0     0       0       H   
                  12          8532        1     0     0       0       H   
...                                      ..   ...   ...     ...     ...   
20240119_AB95_g0  273         115326      1     0     0       0       H   
                  274         115589      1     0     0       0       H   
                  275         115794      1     0     0       0       H   
                  276         115999      1     0     0       0       H   
                  278         116439      1     0     0       0       H   

                                         photometryLhemi  photometryRhemi  \
SessionName       TrialNumber Timestamp                                     
20231206_AB100_g0 1           125               0.092250        -0.179976   
                  4           2044              0.198273         0.097076   
                  6           3394              0.122207        -1.662720   
                  8           4949             -0.637294        -0.146852   
                  12          8532             -0.113219        -0.053371   
...                                                  ...              ...   
20240119_AB95_g0  273         115326           -0.441966         0.122596   
                  274         115589           -0.065142         0.917363   
                  275         115794           -0.846353        -1.354794   
                  276         115999            0.146098        -1.516067   
                  278         116439            0.658653        -1.015172   

                                         lick_1  lick_2  lick_3  ...  go_hit  \
SessionName       TrialNumber Timestamp                          ...           
20231206_AB100_g0 1           125             0       0       0  ...       1   
                  4           2044            0       0       0  ...       1   
                  6           3394            0       0       0  ...       1   
                  8           4949            0       0       0  ...       1   
                  12          8532            0       0       0  ...       1   
...                                         ...     ...     ...  ...     ...   
20240119_AB95_g0  273         115326          0       0       0  ...       1   
                  274         115589          0       0       0  ...       1   
                  275         115794          0       0       0  ...       1   
                  276         115999          0       0       0  ...       1   
                  278         116439          0       0       0  ...       1   

                                         go_miss  nogo_cr  nogo_fa  lick_1R  \
SessionName       TrialNumber Timestamp                                       
20231206_AB100_g0 1           125              0        0        0        0   
                  4           2044             0        0        0        0   
                  6           3394             0        0        0        0   
                  8           4949             0        0        0        0   
                  12          8532             0        0        0        0   
...                                          ...      ...      ...      ...   
20240119_AB95_g0  273         115326           0        0        0        0   
                  274         115589           0        0        0        0   
                  275         115794           0        0        0        0   
                  276         115999           0        0        0        0   
                  278         116439           0        0        0        

## Load your fitting paramaters and set up your train/test data

In [18]:
config_file = os.path.join(project_path, 'config.yaml')
config = utils.load_config(config_file)

In [19]:
# edit the predictors and shift bounds in the config file to the ones that I would like
config['glm_params']['predictors'] = predictor_list
config['glm_params']['predictors_shift_bounds_default'] = predictors_shift_bounds_list
config['glm_params']['response'] = response
config['glm_params']['regression_type'] = regression_type
config['glm_params']['glm_keyword_args']['elasticnet']['alpha'] = alpha
config['glm_params']['glm_keyword_args']['elasticnet']['l1_ratio'] = l1_ratio

# save back to config file
cfg_file = os.path.join(project_dir, project_path, "config.yaml")
utils.save_to_yaml(config, cfg_file)

'e:\\Emily\\sabatini-glm-workflow\\projects\\PV_combinedCatchx4_Lhemi_bestFit\\config.yaml'

#### Shift responses and predictors. If you do not want to shift your predictors by an amount you set, feel free to comment out the entire "predictors_shift_bounds" in config.yaml. We will then use the default set when we created the config file.

In [20]:
#this is the shift operation - remember to use df_source (that has the predictors, rather than df)
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df_source)
print('Your dataframe was shifted using: {}'.format(shifted_params))

### Create your test/train datasets

In [ ]:
X_train,X_test, y_train, y_test = glm_fit.split_data(df_predictors_shift, response_shift, config)

print('Training data has {} rows and {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('Testing data has {} rows and {} columns'.format(X_test.shape[0], X_test.shape[1]))

## Now, we're ready to run our GLM!

### We have three different options: LinearRegression, ElasticNet, and Ridge.

#### ElasticNet: This is a linear regression model trained with both L1 and L2 prior as regularizer. This combination allows for learning a sparse model where few of the weights are non-zero like Lasso, while still maintaining the regularization properties of Ridge. 

#### Ridge: This is a linear regression model trained with L2 prior as regularizer. This is the standard Linear Regression you're familiar with.

#### LinearRegression: This is a standard linear regression model using the OLS method.

#### You can choose which model you would like to use by changing the "regression_type" field in the config.yaml file. You can also change the hyperparameters of the model in the glm_keyword_args field and run cross-validation to find the best hyperparameters. Just remember to change the relevant terms to lists in the config.yaml file. To run cross-validation, you can pass in the optional argument `cross_validatation=True` to the fit method and add `best_params` to your returned outputs.

In [ ]:
model, y_pred, score, beta, intercept = glm_fit.fit_glm(config, X_train, X_test, y_train, y_test, cross_validation=False)
print('Your model can account for {} percent of your data'.format(score*100))

## Save your outputs

In [ ]:
#Create your model dictonary, this should include all the information you wish to save
model_dict = {'model': model,
              'model_type': config['glm_params']['regression_type'],
              'y_pred': y_pred,
              'score': score,
              'beta': beta,
              'intercept': intercept,
              }

#Save your model dictionary
glm_fit.save_model(model_dict, config)

In [ ]:
#save model to text file
model_save_path = config['Project']['project_path'] + '/results/model_results'
with open(model_save_path, 'w') as f:
    print(model_dict, file = f)

model_dict

In [ ]:
#load your model dictionary if you already have it
import pickle
model_path = config['Project']['project_path'] + '/models'
model_name = project_name + '_model.pkl'
model_full_path = os.path.join(model_path, model_name)
with open(model_full_path, 'rb') as f:
    model_dict = pickle.load(f)

In [ ]:
model = model_dict['model']
beta = model_dict['beta']

## Generate and save figures and results.
The following requires non-sparse data. If you have sparse data, you will need to re-run `shift_predictors` with the `sparse` argument set to `False`.

`plot_and_save` will save scatter plots of the predicted vs actual responses and the residuals and your beta coefficients. 

`plot_betas` will only *plot* the beta coefficients. 

`plot_aligned_dataStream` will plot the aligned data stream (e.g. aligned input data). You will need to run the `align_dataStream` function before running this plot.

`plot_actual_v_reconstructed` will plot the actual vs reconstructed responses. You will need to run the `align_reconstructed_dataStream` function before running this plot.


In [ ]:
#first - you need to "desparsify" the data, in order to plot it - set sparsify back to false
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df_source, sparsify=False)
print('Your dataframe was shifted using: {}'.format(shifted_params))

In [ ]:
X_train,X_test, y_train, y_test = glm_fit.split_data(df_predictors_shift, response_shift, config)

print('Training data has {} rows and {} columns'.format(X_train.shape[0], X_train.shape[1]))
print('Testing data has {} rows and {} columns'.format(X_test.shape[0], X_test.shape[1]))

In [ ]:
glm_fit.plot_and_save(config, y_pred, y_test, beta, df_predictors_shift)

In [ ]:
save_path = os.path.join(project_path, 'results')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

#retrieve predicted vs actual responses 
y_pred_all = model.predict(df_predictors_shift)
y_actual = response_shift
y_actual = y_actual.to_numpy()

#plot y_actual and y_pred on same graph for an example section of data
plt.plot(y_actual[13000:14000], color = 'b', lw = 0.5, label = 'actual data')
plt.plot(y_pred_all[13000:14000], color = 'r', lw = 0.5, label = 'predicted data')
plt.legend(loc="upper right")
plt.title("Overlay of actual data and GLM model predicted data")
plt.xlabel("Timestamps")
plt.ylabel("Response")

fig_save_path_png = os.path.join(project_dir, project_name, 'results\\overlay.png')
fig_save_path_pdf = os.path.join(project_dir, project_name, 'results\\overlay.pdf')
plt.savefig(fig_save_path_png)
plt.savefig(fig_save_path_pdf)

plt.show()

In [ ]:
import shutil

# Create new directory for beta plots
save_dir = 'betas'
parent_dir = config['Project']['project_path'] + '/results/'
save_dir_path = os.path.join(parent_dir, save_dir)

# Check if the directory already exists
if not os.path.exists(save_dir_path):
    # Create the directory
    os.makedirs(save_dir_path)
    print("Directory created successfully!")
else:
    print("Directory already exists - will be overwritten!")
    #overwrite the directory
    shutil.copytree(save_dir_path, save_dir_path, dirs_exist_ok = True)
    
save_path = save_dir_path

utils.plot_betas(config, beta, df_predictors_shift, shifted_params, save=None, save_path=None)

### Align the data and plot the actual and reconstructed responses (e.g. predicted y) against the true responses (e.g. neural responses) for each prediction. 

In [ ]:
# Align your actual data
aligned_dataStream = utils.align_dataStream(config, df_source, shifted_params)

# Create new directory for aligned data plots
save_dir = 'aligned'
parent_dir = config['Project']['project_path'] + '/results/'
save_dir_path = os.path.join(parent_dir, save_dir)

# Check if the directory already exists
if not os.path.exists(save_dir_path):
    # Create the directory
    os.makedirs(save_dir_path)
    print("Directory created successfully!")
else:
    print("Directory already exists - will be overwritten!")
    #overwrite the directory
    shutil.copytree(save_dir_path, save_dir_path, dirs_exist_ok = True)

save_path = save_dir_path

# Plot aligned data
utils.plot_aligned_dataStream(aligned_dataStream, config, save=True, save_path=save_path, reconstructed=False)

In [ ]:
# Reconstruct your signal from your X-inputs
recon_dataStream = utils.align_reconstructed_dataStream(config, 
                                                        df_source, df_predictors_shift,
                                                         shifted_params, model)

# Create new directory for reconstructed data plots
save_dir = 'recon'
parent_dir = config['Project']['project_path'] + '/results/'
save_dir_path = os.path.join(parent_dir, save_dir)

# Check if the directory already exists
if not os.path.exists(save_dir_path):
    # Create the directory
    os.makedirs(save_dir_path)
    print("Directory created successfully!")
else:
    print("Directory already exists - will be overwritten!")
    #overwrite the directory
    shutil.copytree(save_dir_path, save_dir_path, dirs_exist_ok = True)

save_path = save_dir_path

# Plot reconstructed data
utils.plot_aligned_dataStream(recon_dataStream, config, save=True, save_path=save_path, reconstructed=True)

In [ ]:
# Create new directory for actual vs reconstructed data plots
save_dir = 'actualVrecon'
parent_dir = config['Project']['project_path'] + '/results/'
save_dir_path = os.path.join(parent_dir, save_dir)

# Check if the directory already exists
if not os.path.exists(save_dir_path):
    # Create the directory
    os.makedirs(save_dir_path)
    print("Directory created successfully!")
else:
    print("Directory already exists - will be overwritten!")
    #overwrite the directory
    shutil.copytree(save_dir_path, save_dir_path, dirs_exist_ok = True)

save_path = save_dir_path

# Plot actual vs reconstructed
utils.plot_actual_v_reconstructed(config, aligned_dataStream, recon_dataStream, save=True, save_path=save_path)

## Additional validation: 

In addition to using k-folds cross-validation, you can also use the `leave_one_out_cross_val` method to validate your model. This method will leave one predictor out and fit the model on the remaining predictors. A `model_list` will be returned with the `model` and `predictions` for each left out predictor.

In [ ]:
##NOTE: Cannot be sparse array for now
model_list = glm_fit.leave_one_out_cross_val(config, X_train, X_test, y_train, y_test, plot=True)

#Save your model_list
import pickle
LO_CV_path = (config['Project']['project_path'] + '/models/LO_CV_models.pkl')
with open(LO_CV_path, 'wb') as f:
        pickle.dump(model_list, f)

#save model_list to text file
model_list_save_path = config['Project']['project_path'] + '/results/leave1Out_model_results'
with open(model_list_save_path, 'w') as f:
    print(model_list, file = f)

model_list

In [ ]:
#predictors temp - removing any reward
#predictors_temp=['nogo', 'lick_1','last_lick', 'lick_non1-3']
predictors_temp=['nogo', 'lick_3', 'lick_3C', 'last_lick', 'lick_non1-3']
X_train_temp = X_train[predictors_temp]
X_test_temp = X_test[predictors_temp]

model_temp, y_pred, score, beta, intercept = glm_fit.fit_glm(config, X_train_temp, X_test_temp, y_train, y_test)

#save model_list to text file
model_temp_save_path = config['Project']['project_path'] + '/results/leaveRewardOut_model_results'
with open(model_temp_save_path, 'w') as f:
    print(score, file = f)

score

### Validating train and test performance:
The following can be used to assess train and test performance. 
This should be done after you have shifted the data, these cells are repeated below for clarity.

In [ ]:
response_shift, df_predictors_shift, shifted_params = glm_fit.shift_predictors(config, df, sparsify=False)
print('Your dataframe was shifted using: {}'.format(shifted_params))

#### Below, you can fit the model with various train/test splits. This will *not* vary the test size, but instead, will vary the IDs used for training and testing.

In [ ]:
#create multiple train-test splits and fit the model
#use AFTER you shift the data (NOT SPLIT) but BEFORE you fit the model

from sklearn.model_selection import ShuffleSplit

n_splits = 3 #set the number of splits you want here
test_size = 0.2 #train test ratio.. this is NO longer pulling from config file
shuffle_split = ShuffleSplit(n_splits=n_splits, test_size=test_size)

X = df_predictors_shift
y = response_shift

#create train-test splits
train_list = []
test_list = []
for i, (train_index, test_index) in enumerate(shuffle_split.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    train_list.append((X_train, y_train))
    test_list.append((X_test, y_test))


In [ ]:
#run the model on each train-test split and plot
results = []
for i, (X_train, y_train) in enumerate(train_list):
    print(f"Fold {i+1}:")
    X_test, y_test = test_list[i]
    model, y_pred, score, beta, intercept = glm_fit.fit_glm(config, X_train, X_test, y_train, y_test)
    #calculate train score
    #fetch regression type and score metric from config
    regression_type = config['glm_params']['regression_type'].lower()
    score_metric = config['glm_params']['glm_keyword_args'][regression_type]['score_metric']
    y_train_pred = model.predict(X_train)
    if score_metric == 'r2':
        train_score = model.score(X_train, y_train)
    elif score_metric == 'mse':
        train_score = glm_fit.calc_mse(y_train, y_train_pred)
    print(f"Train score: {train_score}")
    print(f"Test score: {score}")
    results.append({'n_fold': i+1, 'Train_score': train_score, 'Test_score': score})

#plot boxplot of results with confidence intervals
import matplotlib.pyplot as plt
results_df = pd.DataFrame(results)
results_df.plot(kind='box', y=['Train_score', 'Test_score'])
plt.show()

#### Below, you can fit the model with various train/test splits while doing the LOOCV analysis. This will *not* vary the test size, but instead, will vary the IDs used for training and testing. The results will be a dictionary with `n_folds`, `predictor_left_out`, and `scores`.

### Shuffle and run the LOOCV

In [ ]:
#Run shuffle split cross validation on leave one out models
from sklearn.model_selection import ShuffleSplit

n_splits = 3 #set the number of splits you want here
test_size = 0.2 #train test ratio.. this is NO longer pulling from config file
shuffle_split = ShuffleSplit(n_splits=n_splits, test_size=test_size)

X = df_predictors_shift
y = response_shift

#create train-test splits
train_list = []
test_list = []
for i, (train_index, test_index) in enumerate(shuffle_split.split(X)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    train_list.append((X_train, y_train))
    test_list.append((X_test, y_test))

#run the model on each train-test split
results_LOOCV = []
for i, (X_train, y_train) in enumerate(train_list):
    print(f"Fold {i+1}:")
    X_test, y_test = test_list[i]
    model_list = glm_fit.leave_one_out_cross_val(config, X_train, X_test, y_train, y_test, plot=False)
    results_LOOCV.append({'n_fold': i+1, 'models': model_list})

In [ ]:
## FOR PLOTTING. WILL PARSE THE RESULTS AND PLOT THEM

results_by_predictor = {}

# Fetch test and train scores from each model in the model_list with fold number and predictor left out
for i, result in enumerate(results_LOOCV):
    print(f"Processed fold {i+1}")
    model_list = result['models']
    for j, model_dict in enumerate(model_list):
        train_score = model_dict['train_score']
        score = model_dict['test_score']
        predictor_left_out = model_dict['predictor_left_out']
        
        # Check if the predictor_left_out already exists in the dictionary
        if predictor_left_out not in results_by_predictor:
            results_by_predictor[predictor_left_out] = {'predictor_left_out': predictor_left_out, 'Fold_scores': []}
        
        # Append the scores for the current fold to the corresponding predictor_left_out entry in the dictionary
        results_by_predictor[predictor_left_out]['Fold_scores'].append({'n_fold': i+1, 'Test_score': score, 'Train_score': train_score})

#make boxplot of results by predictor for train and test scores
import seaborn as sns
all_scores = []

# Iterate through results_by_predictor to populate all_scores for easy plotting
for key, value in results_by_predictor.items():
    predictor = key
    fold_scores = value['Fold_scores']
    for fold_score in fold_scores:
        train_score = fold_score['Train_score']
        test_score = fold_score['Test_score']
        all_scores.append({'Predictor_left_out': predictor, 'Score_type': 'Train_score', 'Score': train_score})
        all_scores.append({'Predictor_left_out': predictor, 'Score_type': 'Test_score', 'Score': test_score})

# Convert the list of dictionaries to a dataframe
all_scores_df = pd.DataFrame(all_scores)

# Plotting the box plot
plt.figure(figsize=(10, 6))
boxplot = sns.boxplot(data=all_scores_df, x='Predictor_left_out', y='Score', hue='Score_type', palette='coolwarm')
boxplot.set_title('Train and Test Scores by Predictor')

fig_save_path_png = os.path.join(project_dir, project_name, 'results\\trainAndTestbyPred.png')
fig_save_path_pdf = os.path.join(project_dir, project_name, 'results\\trainAndTestbyPred.pdf')
plt.savefig(fig_save_path_png)
plt.savefig(fig_save_path_pdf)

plt.show()